In [1]:
import cv2
import numpy as np
from pandas import Series, DataFrame

In [2]:
###########################
### Spectral Analysis
###########################

try:
    HISTOGRAM_BITS_PER_COLOR
except:
    HISTOGRAM_BITS_PER_COLOR = 3
    
NUM_BINS = 1 << HISTOGRAM_BITS_PER_COLOR
COLOR_SPECTRUM = 2 ** (3 * HISTOGRAM_BITS_PER_COLOR) + 1

ADAPTIVE_HISTOGRAM_THRESHOLD = True
HISTOGRAM_SUGGESTED_THRESHOLD = 0
HISTOGRAM_COUNT_THRESHOLD = HISTOGRAM_SUGGESTED_THRESHOLD

if HISTOGRAM_BITS_PER_COLOR == 1:
    HISTOGRAM_CEILING_THRESHOLD = 1500
    HISTOGRAM_FLOOR_THRESHOLD = 75
elif HISTOGRAM_BITS_PER_COLOR == 2:
    HISTOGRAM_CEILING_THRESHOLD = 1500
    HISTOGRAM_FLOOR_THRESHOLD = 50
else:
    HISTOGRAM_CEILING_THRESHOLD = 1500
    HISTOGRAM_FLOOR_THRESHOLD = 15

SHOW_DUPES = False
DEDUPE_REGIONS = True
MIN_DEDUPE_REGIONS = 5
REGION_MERGE_PIXEL_DISTANCE = 20
MIN_REGION_SCORE = 0    

class BinStats:
    spacing = 0
    min = 0
    max = 0
    
class Bin:
    r = BinStats()
    g = BinStats()
    b = BinStats()
    
def InitBin(image):
    bin = Bin()
    
    print 'InitBin:'
    r = image[:,:,0]
    bin.r.min = np.min(r)
    bin.r.max = np.max(r)
    bin.r.spacing = 1 + (bin.r.max - bin.r.min)/NUM_BINS
    print "min_r=%i, max_r=%i, bin_spacing.r=%i" % (bin.r.min, bin.r.max, bin.r.spacing)
    
    g = image[:,:,1]
    bin.g.min = np.min(g)
    bin.g.max = np.max(g)
    bin.g.spacing = 1 + (bin.g.max - bin.g.min)/NUM_BINS
    print "min_g=%i, max_g=%i, bin_spacing.g=%i" % (bin.g.min, bin.g.max, bin.g.spacing)
    
    b = image[:,:,2]
    bin.b.min = np.min(b)
    bin.b.max = np.max(b)
    bin.b.spacing = 1 + (bin.b.max - bin.b.min)/NUM_BINS
    print "min_b=%i, max_b=%i, bin_spacing.b=%i" % (bin.b.min, bin.b.max, bin.b.spacing)
    
    return bin
    
def InitLUT(bin):
    r_lut = np.zeros(256)
    g_lut = np.zeros(256)
    b_lut = np.zeros(256)
    
    for i in range(256):
        r_lut[i] = (i - bin.r.min) / bin.r.spacing
        if r_lut[i] >= NUM_BINS:
            r_lut[i] = NUM_BINS - 1
            
        g_lut[i] = (i - bin.g.min) / bin.g.spacing
        if g_lut[i] >= NUM_BINS:
            g_lut[i] = NUM_BINS - 1
            
        b_lut[i] = (i - bin.b.min) / bin.b.spacing
        if b_lut[i] >= NUM_BINS:
            b_lut[i] = NUM_BINS - 1
    
#    print r_lut
#    print g_lut
#    print b_lut
    
    return r_lut,g_lut,b_lut

def quantize(image, lut):
    height,width,depth = image.shape
    print "image height=%i,width=%i,depth=%i" % (height,width,depth)
    
    r_lut,g_lut,b_lut = lut
    
    qimage = np.zeros((height,width,depth),dtype='uint8')
    qimage[:,:,0] = r_lut[image[:,:,0]]
    qimage[:,:,1] = g_lut[image[:,:,1]]
    qimage[:,:,2] = b_lut[image[:,:,2]]
        
#    for h in range(height):
#        for w in range(width):
#            qimage_r[h][w] = r_lut[r[h][w]]
#            qimage_g[h][w] = g_lut[g[h][w]]
#            qimage_b[h][w] = b_lut[b[h][w]]

    return qimage

def is_zero_pixel(pixel):
    r,g,b = pixel
    
    return r == 0 and g == 0 and b == 0

def unquantize(qimage, bin):
    height,width,depth = qimage.shape
    print "qimage height=%i,width=%i,depth=%i" % (height,width,depth)
    
    dimage = np.zeros((height,width,depth),dtype='uint8')
    dimage[:,:,0] = qimage[:,:,0]*bin.r.spacing + bin.r.min
    dimage[:,:,1] = qimage[:,:,1]*bin.g.spacing + bin.g.min
    dimage[:,:,2] = qimage[:,:,2]*bin.b.spacing + bin.b.min

# Old way is super slow, in the order of several seconds
#    for h in range(height):
#        for w in range(width):
#            if is_zero_pixel(qimage[h][w]):
#                dimage[h][w] = qimage[h][w]
#            else:
#                dimage_r[h][w] = qimage_r[h][w] * bin.r.spacing + bin.r.min
#                dimage_g[h][w] = qimage_g[h][w] * bin.g.spacing + bin.g.min
#                dimage_b[h][w] = qimage_b[h][w] * bin.b.spacing + bin.b.min
    
    return dimage

def BuildPixel(rgb):
    r,g,b = rgb
    
    return (r << 2*HISTOGRAM_BITS_PER_COLOR) + (g << HISTOGRAM_BITS_PER_COLOR) + b
    
def BuildHistogram(qimage):
    height,width,depth = qimage.shape
    hist = np.zeros(COLOR_SPECTRUM)

    r = qimage[:,:,0]
    g = qimage[:,:,1]
    b = qimage[:,:,2]
    pixel= BuildPixel((r,g,b))
    
    flat = pixel.reshape(pixel.shape[0]*pixel.shape[1])
    hist = Series(flat).value_counts()
    hist[hist > 10000] = 10000
    
    # Fill in the missing indexes with zero values
    for i in range(COLOR_SPECTRUM):
        try:
            hist[i]
        except:
            hist[i] = 0

    hist = hist.sort_index(axis=0)

#    for h in range(height):
#        for w in range(width):
#            rgb = qimage[h,w]
#            pixel = BuildPixel(rgb)
#            hist[pixel] += 1
#            hist[pixel] = min(hist[pixel], 10000)
    
    return hist.values

def HistogramThreshold(qimage, hist):
    global HISTOGRAM_COUNT_THRESHOLD
    height,width,depth = qimage.shape
    
    max_threshold = HISTOGRAM_SUGGESTED_THRESHOLD
    if ADAPTIVE_HISTOGRAM_THRESHOLD:
        max_threshold = max(min(min(hist[np.nonzero(hist)]) / 100, HISTOGRAM_CEILING_THRESHOLD/100) * 100, HISTOGRAM_FLOOR_THRESHOLD)
        
    threshold = min(hist, key=lambda x:int(abs(x-max_threshold))) + 1
    HISTOGRAM_COUNT_THRESHOLD = threshold
    
    mask = np.zeros((height,width,depth),dtype='uint8')
#    mask_r = mask[:,:,0]
#    mask_g = mask[:,:,1]
#    mask_b = mask[:,:,2]
    
#    for h in range(height):
#        for w in range(width):
#            pixel = BuildPixel(qimage[h,w])
#            if hist[pixel] > HISTOGRAM_COUNT_THRESHOLD:
#                mask_r[h,w] = mask_g[h,w] = mask_b[h,w] = 0
#            else:
#                mask[h,w] = qimage[h,w]

    # Use the super-cool-and-fast pandas DataFrame to subtract out the background pixels
    r = qimage[:,:,0]
    g = qimage[:,:,1]
    b = qimage[:,:,2]
    pixel= BuildPixel((r,g,b))
    flat_pixel = pixel.reshape(pixel.shape[0]*pixel.shape[1])

    isBackground = hist[flat_pixel] > HISTOGRAM_COUNT_THRESHOLD

    flat_r = qimage[:,:,0].reshape(height*width)
    flat_g = qimage[:,:,1].reshape(height*width)
    flat_b = qimage[:,:,2].reshape(height*width)

    df = DataFrame({'pixel':flat_pixel,'r':flat_r,'g':flat_g,'b':flat_b,'is_background':isBackground})

    tt = df['pixel'].groupby(df['is_background'])
    if True in tt.indices:
        background_ix = tt.indices[True]
        df.ix[background_ix] = 0
    
    mask[:,:,0] = df['r'].values.reshape(height,width)
    mask[:,:,1] = df['g'].values.reshape(height,width)
    mask[:,:,2] = df['b'].values.reshape(height,width)

    return mask

In [3]:
########################
### Spatial Analysis
########################

MAX_REGIONS = 4000
REGION_UNKNOWN = -2
REGION_NONE = -1

MAX_REGION_AREA = 0.25
MIN_REGION_AREA = 0.04
MAX_REGION_SIZE = 0.5
MIN_REGION_SIZE = 0.2

try:
    MIN_REGION_SCORE
except NameError:
    MIN_REGION_SCORE = -1

class ARegion:
    def __init__(self):
        self.minx = 0
        self.maxx = 0
        self.miny = 0
        self.maxy = 0
        self.score = 0
        self.size = 0
        self.avg_pixel = 0
        self.active = True
    
class Regions:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.num_regions = 0
        self.catalog = []
        
        self.data = np.zeros((self.height, self.width),dtype='int')
#        for h in range(self.height):
#            for w in range(self.width):
#                self.data[h,w] = REGION_UNKNOWN

        self.data = self.data.reshape(self.height*self.width)
        self.data = self.data + REGION_UNKNOWN
        self.data = self.data.reshape(self.height, self.width)

def FindRegion(regions, w, h):
    region_merge = REGION_MERGE_PIXEL_DISTANCE #regions.width / 100
    
    for yofs in range(-1*region_merge, 1):
        for xofs in range(-1*region_merge, region_merge+1):
            if yofs+h < 0:
                continue
            if xofs+w < 0:
                continue
            if xofs+w >= regions.width:
                continue
            
            if regions.data[yofs+h][xofs+w] >= 0:
                return regions.data[yofs+h][xofs+w]
            
    for xofs in range(-1*region_merge, 0):
        if xofs+w  < 0:
            continue
        if regions.data[h][xofs+w] >= 0:
            return regions.data[h][xofs+w]
        
    return REGION_NONE
    
def AssignRegions(mask):
    height,width,depth = mask.shape
    regions = Regions(width, height)

    r = mask[:,:,0]
    g = mask[:,:,1]
    b = mask[:,:,2]
    pixel = BuildPixel((r,g,b))
    nz_pixels = np.nonzero(pixel)

    # Initialize region matrix for all empty pixels with REGION_NONE
    pixel = pixel.reshape(height * width)
    is_empty_pixel = pixel == 0
    flat_data = regions.data.reshape(height * width)
    df = DataFrame({'data':flat_data, 'is_empty':is_empty_pixel})
    tt = df['data'].groupby(df['is_empty'])
    if True in tt.indices:
        background_ix = tt.indices[True]
        df.ix[background_ix] = REGION_NONE
    regions.data = df['data'].values.reshape(height,width)

    # for all the foreground pixels (non-zero) assign a region
    for i in range(len(nz_pixels[0])):
        h = nz_pixels[0][i]
        w = nz_pixels[1][i]

        r = FindRegion(regions, w, h)
        if r == REGION_NONE:
            r = regions.num_regions
            regions.num_regions += 1

            aregion = ARegion()
            aregion.minx = w
            aregion.maxx = w
            aregion.miny = h
            aregion.maxy = h
            aregion.size = 1

            regions.catalog.append(aregion)
        else:
            regions.catalog[r].minx = min(regions.catalog[r].minx, w)
            regions.catalog[r].miny = min(regions.catalog[r].miny, h)
            regions.catalog[r].maxx = max(regions.catalog[r].maxx, w)
            regions.catalog[r].maxy = max(regions.catalog[r].maxy, h)
            regions.catalog[r].size = (1 + regions.catalog[r].maxx - regions.catalog[r].minx) * (1 + regions.catalog[r].maxy - regions.catalog[r].miny)

        regions.data[h,w] = r

    return regions
    
def _AssignRegions(mask):
    height,width,depth = mask.shape
    
    regions = Regions(width, height)
    for h in range(height):
        for w in range(width):
            if regions.data[h,w] == REGION_UNKNOWN:
                if is_zero_pixel(mask[h,w]):
                    regions.data[h,w] = REGION_NONE
                    continue
                
                if regions.num_regions == MAX_REGIONS:
                    return regions
                    
                r = FindRegion(regions, w, h)
                if r == REGION_NONE:
                    r = regions.num_regions
                    regions.num_regions += 1
                    
                    aregion = ARegion()
                    aregion.minx = w
                    aregion.maxx = w
                    aregion.miny = h
                    aregion.maxy = h
                    aregion.size = 1
                    
                    regions.catalog.append(aregion)
                else:
                    regions.catalog[r].minx = min(regions.catalog[r].minx, w)
                    regions.catalog[r].miny = min(regions.catalog[r].miny, h)
                    regions.catalog[r].maxx = max(regions.catalog[r].maxx, w)
                    regions.catalog[r].maxy = max(regions.catalog[r].maxy, h)
                    regions.catalog[r].size = (1 + regions.catalog[r].maxx - regions.catalog[r].minx) * (1 + regions.catalog[r].maxy - regions.catalog[r].miny)
                        
                regions.data[h,w] = r
                
    return regions

def RemoveRegion(regions, i):
    regions.catalog[i].minx = 0
    regions.catalog[i].miny = 0
    regions.catalog[i].maxx = 0
    regions.catalog[i].maxy = 0
    
def RegionTooLarge(regions, i):
    xlen = regions.catalog[i].maxx - regions.catalog[i].minx
    ylen = regions.catalog[i].maxy - regions.catalog[i].miny
    
    return (xlen*ylen > MAX_REGION_AREA) or (xlen > MAX_REGION_SIZE) or (ylen > MAX_REGION_SIZE)

def PruneLargeRegions(regions):
    for i in range(regions.num_regions):
        if RegionTooLarge(regions, i):
            RemoveRegion(regions, i)

def RegionTooSmall(regions, i):
    xlen = regions.catalog[i].maxx - regions.catalog[i].minx
    ylen = regions.catalog[i].maxy - regions.catalog[i].miny
    
    return (xlen*ylen < MAX_REGION_AREA) or (xlen < MAX_REGION_SIZE) or (ylen < MAX_REGION_SIZE)
    
def PruneSmallRegions(regions):
    for i in range(regions.num_regions):
        if RegionTooSmall(regions, i):
            RemoveRegion(regions, i)

def ScoreOneRegion(aregion, qimage, hist):
    score = 0
    count = 0
    r_count = g_count = b_count = 0
    rgb_count = 0
    
    for h in range(aregion.miny, aregion.maxy):
        for w in range(aregion.minx, aregion.maxx):
            r,g,b = qimage[h,w]
            pixel = BuildPixel((r,g,b))
            
            if hist[pixel] > HISTOGRAM_COUNT_THRESHOLD:
                continue
                
            r_count += r
            g_count += g
            b_count += b
            rgb_count += 1
    
            distance = HISTOGRAM_COUNT_THRESHOLD - hist[pixel]
            count += 1
            score += distance
            
    if count == 0:
        return 0
    else:
        r_count /= rgb_count
        g_count /= rgb_count
        b_count /= rgb_count
        aregion.avg_pixel = BuildPixel((r_count,g_count,b_count))

        return min(1000, int((1000*score)/(count*HISTOGRAM_COUNT_THRESHOLD)))
    
def ScoreRegions(regions, qimage, hist):
    for aregion in regions.catalog:
        aregion.score = ScoreOneRegion(aregion, qimage, hist)
        
def PruneRegionsByThreshold(regions):
    global MIN_REGION_SCORE
    EVALUATE_PREVIOUS_INDEX = 5
    NEAREST_FACTOR = 100
    
    total_regions = len(regions.catalog)
    if total_regions > 0:
        array = np.sort(np.array([regions.catalog[i].score for i in range(regions.num_regions)]))
        nz = np.nonzero(array)
        floor_score = average_score = ceiling_score = 0
#        if len(nz[0]) > 0:
#            floor_score = array[nz[0][0]]
#            average_score = np.mean(array)
#            median_score = np.median(nz[0])
#            ceiling_score = array[nz[0][-1]]
        
#        print "  Min Region Score: %i" % floor_score
#        print "  Average Region Score: %i" % average_score
#        print "  Median Region Score: %i" % median_score
#        print "  Max Region Score: %i" % ceiling_score
        
        print "Prune Common Regions (below threshold):"
        min_score = MIN_REGION_SCORE
        if min_score == -1:
            scores = np.sort([region.score for region in regions.catalog])
            score = scores[0]
            if len(scores) > EVALUATE_PREVIOUS_INDEX:
                score = scores[-EVALUATE_PREVIOUS_INDEX]
            min_score = (score/NEAREST_FACTOR) * NEAREST_FACTOR - 1 #(np.median(nz) + np.max(nz))/2
            
            print "  Evaluate Previous Index: %i" % EVALUATE_PREVIOUS_INDEX
            print "  Nearest Factor: %i" % NEAREST_FACTOR
        print "  Min Score: %i" % min_score
    
        pruned_regions = []
        for aregion in regions.catalog:
            if aregion.score > min_score:
                pruned_regions.append(aregion)
        regions.catalog = pruned_regions
        regions.num_regions = len(pruned_regions)
        print "  Regions Above Threshold (Active): %i" % regions.num_regions

def PruneRegionsByMean(regions):
#     scores = []
#     avg_pixels = []
#     active = []
    
#     for aregion in regions.catalog:
#         scores.append(aregion.score)
#         avg_pixels.append(aregion.avg_pixel)
#         active.append(aregion.active)
            
#     df = DataFrame({'Region Score':scores,'Region Pixel Avg':avg_pixels, 'Region Active':active})
#     mean = df['Region Pixel Avg'].mean()
    
    mean = np.mean([aregion.avg_pixel for aregion in regions.catalog])
    mean = 0 if np.isnan(mean) else mean
    
    active_regions = []
    index = 0
    for aregion in regions.catalog:
        if aregion.avg_pixel >= mean:
            aregion.active = False
        else:
            active_regions.append(index)
        index += 1
            
    print "Prune Regions Exceeding Mean:"
    print "  Mean: %i" % mean
    print "  Regions Below Mean (Active): %i" % len(active_regions)
    print "  [id]: " + str(sorted(active_regions))
    
def PruneRegionsByMedian(regions):
    median = np.median([aregion.avg_pixel for aregion in regions.catalog])
    median = 0 if np.isnan(median) else median
    
    active_regions = []
    index = 0
    for aregion in regions.catalog:
        if aregion.avg_pixel >= median:
            aregion.active = False
        elif aregion.active:
            active_regions.append(index)
        index += 1
            
    print "Prune Regions Exceeding Median:"
    print "  Median: %i" % median
    print "  Regions Below Median (Active): %i" % len(active_regions)
    print "  [id]: " + str(sorted(active_regions))
    
def PruneRegionsByDupe(regions):
    like_regions = np.zeros(1000)
    DDUPE_ACCURACY_BY = 10
    
    print "Prune Duplicated Regions:"
    print "  Accuracy: %i" % DDUPE_ACCURACY_BY
    
    active_regions = []
    index = 0
    if DEDUPE_REGIONS == True:
        for aregion in regions.catalog:
            val = ((aregion.score/DDUPE_ACCURACY_BY)*DDUPE_ACCURACY_BY) / aregion.avg_pixel
            like_regions[val] += 1
            if like_regions[val] > 1:
                aregion.active = False
            elif aregion.active:
                active_regions.append(index)
            index += 1
    
    if regions.num_regions > 0 and regions.num_regions <= MIN_DEDUPE_REGIONS:
        active_regions = []
        index = 0
        for aregion in regions.catalog:
            aregion.active = True
            active_regions.append(index)
            index += 1
        print "  Reinstated All Regions Below Count: %i" % MIN_DEDUPE_REGIONS
    
    print "  Regions Deduped (Active): %i" % len(active_regions)
    print "  [id]: " + str(sorted(active_regions))

def PruneRegionsByProximity(regions):
    previous_active_region = None
    radius = REGION_MERGE_PIXEL_DISTANCE * 10
    
    active_regions = []
    index = 0
    for aregion in regions.catalog:
        if aregion.active:
            if len(active_regions) > 0:
                for i in active_regions:
                    region = regions.catalog[i]
                    pregion_centerx = region.maxx #(previous_region.maxx - previous_region.minx)/2
                    pregion_centery = region.maxy #(previous_region.maxy - previous_region.miny)/2
                    aregion_centerx = aregion.maxx #(aregion.maxx - aregion.minx)/2
                    aregion_centery = aregion.maxy #(aregion.maxy - aregion.miny)/2
                    if (aregion_centerx >= max(pregion_centerx-radius,0) and aregion_centerx <= (pregion_centerx+radius)) and \
                       (aregion_centery >= max(pregion_centery-radius,0) and aregion_centery <= (pregion_centery+radius)):
                        aregion.active = False
                        break
                
                if aregion.active:
                    active_regions.append(index)
            else:
                active_regions.append(index)
        index += 1
    
    print "Pruned Clustered Regions:"
    print "  Radius of Separation: %i" % radius
    print "  Regions Isolated (Active): %i" % len(active_regions)
    print "  [id]: " + str(sorted(active_regions))
    
def MarkRegions(image, regions):
    width = regions.width
    height = regions.height
    red = (255,0,0)
    
#    for aregion in regions.catalog:
    scores = []
    avg_pixels = []
    active = []
    mod = []
    div = []
    print '\n'
    print 'Annotating Active Regions: %i' % regions.num_regions
    for i in range(regions.num_regions):
        aregion = regions.catalog[i]
        cv2.rectangle(image, (max(aregion.minx-2, 0), max(aregion.miny-2, 0)), (min(aregion.maxx+2, width-1), min(aregion.maxy+2, height-1)), red)
        if aregion.active:
            cv2.putText(image, str(i), (max(aregion.minx-2, 0), max(aregion.miny-2, 0)), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,0,0),4)
        elif SHOW_DUPES:
            cv2.putText(image, str(i), (max(aregion.minx-2, 0), max(aregion.miny-2, 0)), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),4)
            
        scores.append(aregion.score)
        avg_pixels.append(aregion.avg_pixel)
        active.append(aregion.active)
        mod.append(np.mod(aregion.score,aregion.avg_pixel))
        div.append(aregion.score/aregion.avg_pixel)
    df = DataFrame({'Region Score':scores,'Region Pixel Avg':avg_pixels, 'Region Active':active, 'Mod':mod, 'Div':div})
    df = df.reindex(columns=['Region Score', 'Region Pixel Avg', 'Mod', 'Div', 'Region Active'])
    print df
    if len(df.index) > 0:
        by_score = df['Region Pixel Avg'].groupby([df['Region Active'],df['Region Score']])
        print by_score.value_counts()
        print '\nmean: '
        print df.mean()
        print '\nstd: '
        print df.std()
        print '\n'
        print df['Region Score'].groupby(df['Region Active']).size()

In [4]:
def PlotHistogram_RGB(qimage):
    qr = qimage[:,:,0]
    qg = qimage[:,:,1]
    qb = qimage[:,:,2]

    height,width,depth = qimage.shape
    r_hist = Series(qr.reshape(height*width)).value_counts()
    g_hist = Series(qg.reshape(height*width)).value_counts()
    b_hist = Series(qb.reshape(height*width)).value_counts()

    hist_index = np.arange(2**HISTOGRAM_BITS_PER_COLOR)
    my_dpi = 160
    bar_width = 0.2
    r_index = hist_index
    g_index = hist_index + bar_width
    b_index = hist_index + (bar_width * 2)
    plt.figure(figsize=(1900/my_dpi, 1280/my_dpi), dpi=my_dpi)
    plt.bar(r_index, r_hist, bar_width, color='r')
    plt.bar(g_index, g_hist, bar_width, color='g')
    plt.bar(b_index, b_hist, bar_width, color='b')

    plt.xlabel('Bins')
    plt.ylabel('Count')
    plt.xticks(hist_index + bar_width, hist_index)
    
def PrintSettings():
    settings = \
    'HISTOGRAM_BITS_PER_COLOR = {0}\n'\
    'NUM_BINS = {1}\n'\
    'COLOR_SPECTRUM = {2}\n'\
    'ADAPTIVE_HISTOGRAM_THRESHOLD = {3}\n'\
    'HISTOGRAM_CEILING_THRESHOLD = {4}\n'\
    'HISTOGRAM_FLOOR_THRESHOLD = {5}\n'\
    'SHOW_DUPES = {6}\n'\
    'DEDUPE_REGIONS = {7}\n'\
    'MIN_DEDUPE_REGIONS = {8}\n'\
    'REGION_MERGE_PIXEL_DISTANCE = {9}\n'\
    'MIN_REGION_SCORE = {10}\n'\
    .format(HISTOGRAM_BITS_PER_COLOR,
             NUM_BINS,
             COLOR_SPECTRUM,
             ADAPTIVE_HISTOGRAM_THRESHOLD,
             HISTOGRAM_CEILING_THRESHOLD,
             HISTOGRAM_FLOOR_THRESHOLD,
             SHOW_DUPES,
             DEDUPE_REGIONS,
             MIN_DEDUPE_REGIONS,
             REGION_MERGE_PIXEL_DISTANCE,
             MIN_REGION_SCORE)
    
    if ADAPTIVE_HISTOGRAM_THRESHOLD:
        print 'Adaptive Threshold Enabled \n'
    else:
        print 'Adaptive Threshold Disabled with count set at %i \n' % HISTOGRAM_SUGGESTED_THRESHOLD
        
    print settings